<a href="https://colab.research.google.com/github/Zarak00/Projects/blob/main/Collins_Aerospace/Subscription_Report%20/Subscription_Report%20_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Subscription Report Automated

In [1]:
import pandas as pd
import os
from google.colab import files
import numpy as np

In [2]:
# Creating a folder named "data"
folder_path = '/content/data'  # Specify the folder path
os.makedirs(folder_path, exist_ok=True)


# Upload Subscription Report Here



In [3]:
# Upload the Excel file
uploaded = files.upload()

# Move the uploaded file to the "data" folder
file_name = list(uploaded.keys())[0]
file_path = os.path.join(folder_path, file_name)
os.rename(file_name, file_path)


Saving June 2023 Subscription Report.xlsx to June 2023 Subscription Report.xlsx


In [4]:
df1 = pd.read_excel(file_path, sheet_name="Base Plans")
df1.head(1)

,CUSTOMER_SUB_ID,CUSTOMER,CUST_CTY,CUST_ST,CUST_COUNTRY,SUB_ID,SUB_NM,BILLING_CTY,BILLING_ST,BILLING_COUNTRY,...,FEATURE,TAIL_NUMBER,RATE_CODE_STRT_DTTM,RATE_CODE_END_DTTM,RATE_CODE,ADC_TAIL_ID,RP_BILL_THRU_DTTM,UNIT_PRICE,PRICE_OVERRIDE,DISCOUNT_OVERRIDE
0,141382,Fair Wind Air Charter,Stuart,FL,USA,145720,FAIWINADC01_ADC_DOM,STUART,FL,USA,...,Gold Subscription (Annual) - JAN,N227SV,2013-03-01,NaT,GOLDSAT001_B,14264,2024-02-29 23:59:59,15915.0,NaN,2.0


# Upload Reference Sheet Here

In [5]:
uploaded = files.upload()

Saving Subscription Reference .xlsx to Subscription Reference .xlsx


In [6]:
# Get the uploaded file name
file_name = list(uploaded.keys())[0]

# Read the Excel file
df2 = pd.read_excel(file_name)
df2.head(1)

,Feature,Plan,Frequency,# of Payments,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,A La Carte Access Fee,A La Carte,Monthly,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df1.head(1)

,CUSTOMER_SUB_ID,CUSTOMER,CUST_CTY,CUST_ST,CUST_COUNTRY,SUB_ID,SUB_NM,BILLING_CTY,BILLING_ST,BILLING_COUNTRY,...,FEATURE,TAIL_NUMBER,RATE_CODE_STRT_DTTM,RATE_CODE_END_DTTM,RATE_CODE,ADC_TAIL_ID,RP_BILL_THRU_DTTM,UNIT_PRICE,PRICE_OVERRIDE,DISCOUNT_OVERRIDE
0,141382,Fair Wind Air Charter,Stuart,FL,USA,145720,FAIWINADC01_ADC_DOM,STUART,FL,USA,...,Gold Subscription (Annual) - JAN,N227SV,2013-03-01,NaT,GOLDSAT001_B,14264,2024-02-29 23:59:59,15915.0,NaN,2.0


In [8]:
df1.columns = df1.columns.str.lower()
df2.columns = df2.columns.str.lower()

# Perform the lookup and merge the DataFrames based on the 'feature' column
df_merged = pd.merge(df1, df2[['feature', 'plan']], left_on='feature', right_on='feature', how='left')
df_merged = df_merged.rename(columns={'plan': 'Plans'})
df_merged.head(1)

,customer_sub_id,customer,cust_cty,cust_st,cust_country,sub_id,sub_nm,billing_cty,billing_st,billing_country,...,tail_number,rate_code_strt_dttm,rate_code_end_dttm,rate_code,adc_tail_id,rp_bill_thru_dttm,unit_price,price_override,discount_override,Plans
0,141382,Fair Wind Air Charter,Stuart,FL,USA,145720,FAIWINADC01_ADC_DOM,STUART,FL,USA,...,N227SV,2013-03-01,NaT,GOLDSAT001_B,14264,2024-02-29 23:59:59,15915.0,NaN,2.0,Gold


In [9]:
# Perform the VLOOKUP operation using merge
df_merged = pd.merge(df_merged, df2[['feature', '# of payments']],
                     left_on='feature', right_on='feature', how='left')

df_merged.rename(columns={'# of payments': 'Number of Payments'}, inplace=True)
df_merged.head(1)


,customer_sub_id,customer,cust_cty,cust_st,cust_country,sub_id,sub_nm,billing_cty,billing_st,billing_country,...,rate_code_strt_dttm,rate_code_end_dttm,rate_code,adc_tail_id,rp_bill_thru_dttm,unit_price,price_override,discount_override,Plans,Number of Payments
0,141382,Fair Wind Air Charter,Stuart,FL,USA,145720,FAIWINADC01_ADC_DOM,STUART,FL,USA,...,2013-03-01,NaT,GOLDSAT001_B,14264,2024-02-29 23:59:59,15915.0,NaN,2.0,Gold,1


In [10]:
df_merged['override price'] = df_merged['price_override'].fillna(df_merged['unit_price'] * df_merged['Number of Payments'])

the code assigns the value of 'price_override' to the 'override price' column in the 'df_merged' DataFrame, filling any missing values with the result of multiplying 'unit_price' by 'Number of Payments'.

In [11]:
df_merged['discount price'] = df_merged['override price'] * (1 - df_merged['discount_override'] / 100)


 the code assigns the value of 'override price' multiplied by the difference of 1 and 'discount_override' divided by 100 to the 'discount price' column in the 'df_merged' DataFrame.

In [12]:
df_merged['Annual Price'] = df_merged['discount price'].fillna(df_merged['override price'])



the code assigns the value of 'discount price' to the 'Annual Price' column in the 'df_merged' DataFrame, filling any missing values with the values from the 'override price' column.

In [15]:
df_merged.head(3)

,customer_sub_id,customer,cust_cty,cust_st,cust_country,sub_id,sub_nm,billing_cty,billing_st,billing_country,...,adc_tail_id,rp_bill_thru_dttm,unit_price,price_override,discount_override,Plans,Number of Payments,override price,discount price,Annual Price
0,141382,Fair Wind Air Charter,Stuart,FL,USA,145720,FAIWINADC01_ADC_DOM,STUART,FL,USA,...,14264,2024-02-29 23:59:59,15915.0,NaN,2.0,Gold,1,15915.0,15596.7,15596.7
1,167641,"Hormel Foods Corporate Services, LLC",Austin,MN,USA,167916,HORFOOADC02_ADC_DOM,Austin,MN,USA,...,18994,2023-05-31 23:59:59,49.0,NaN,NaN,Iridium Data,12,588.0,NaN,588.0
2,141382,Fair Wind Air Charter,Stuart,FL,USA,210438,FAIWINADC31_ADC_DOM,STUART,FL,USA,...,39379,2023-12-31 23:59:59,1250.0,NaN,NaN,FDWG,1,1250.0,NaN,1250.0


# Download Report

In [16]:
df_merged.to_excel('Subscription_Report.xlsx', index=False)
